In [1]:
print('c')

c


In [35]:
import os
import pandas as pd
from langchain_core.documents import Document
from dotenv import load_dotenv
from langchain_astradb import AstraDBVectorStore

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq

In [37]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [72]:
import uvicorn
from fastapi import FastAPI ,Request,Form
from fastapi.responses import HTMLResponse
from fastapi.templating import Jinja2Templates
from fastapi.middleware.cors import CORSMiddleware
from fastapi.staticfiles import StaticFiles

In [62]:
from langchain_core.runnables import RunnablePassthrough

from langchain_core.output_parsers import StrOutputParser

from langchain_core.prompts import ChatPromptTemplate


In [ ]:
# Load product data from CSV

'd:\\Customer_Support_System(GenAi-RAG)\\notebook'

In [11]:
csv_path=r'D:\Customer_Support_System(GenAi-RAG)\data\flipkart_product_review.csv'


In [12]:
df=pd.read_csv(csv_path)
df.head(3)

,product_id,product_title,rating,summary,review
0,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Terrific purchase,1-more flexible2-bass is very high3-sound clar...
1,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Terrific purchase,Super sound and good looking I like that prize
2,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Super!,Very much satisfied with the device at this pr...


In [13]:
df.columns

Index(['product_id', 'product_title', 'rating', 'summary', 'review'], dtype='object')

In [14]:
df.shape

(450, 5)

In [22]:
#Transform df data into list of Langchain Document objects.
product_list=[]
for _, row in df.iterrows():
            product_entry={
                "product_name":row['product_title'],
                "product_rating":row['rating'],
                "product_summary":row['summary'],
                "product_review":row['review']
            }
            # print('product_entry :',product_entry)
            product_list.append(product_entry)


documents=[]
for entry in product_list:
        metadata={
                "product_name":entry["product_name"],
                "product_rating":entry["product_rating"],
                "product_summary":entry["product_summary"]}
        doc=Document(page_content=entry["product_review"],metadata=metadata)
        documents.append(doc)

In [18]:
product_list[0]

{'product_name': 'BoAt Rockerz 235v2 with ASAP charging Version 5.0 Bluetooth Headset',
 'product_rating': 5,
 'product_summary': 'Terrific purchase',
 'product_review': "1-more flexible2-bass is very high3-sound clarity is good 4-battery back up to 6 to 8 hour's 5-main thing is fastest charging system is available in that. Only 20 min charge and get long up to 4 hours back up 6-killing look awesome 7-for gaming that product does not support 100% if you want for gaming then I'll recommend you please don't buy but you want for only music then this product is very well for you.. 8-no more wireless headphones are comparing with that headphones at this pric..."}

In [20]:
len(product_list)

450

In [25]:
documents[0]

Document(metadata={'product_name': 'BoAt Rockerz 235v2 with ASAP charging Version 5.0 Bluetooth Headset', 'product_rating': 5, 'product_summary': 'Terrific purchase'}, page_content="1-more flexible2-bass is very high3-sound clarity is good 4-battery back up to 6 to 8 hour's 5-main thing is fastest charging system is available in that. Only 20 min charge and get long up to 4 hours back up 6-killing look awesome 7-for gaming that product does not support 100% if you want for gaming then I'll recommend you please don't buy but you want for only music then this product is very well for you.. 8-no more wireless headphones are comparing with that headphones at this pric...")

In [26]:
len(documents)

450

In [42]:
os.environ["GOOGLE_API_KEY"]=os.getenv("GOOGLE_API_KEY")

In [46]:
collection_name="ecommercedata1"
vstore=AstraDBVectorStore(
            embedding= GoogleGenerativeAIEmbeddings(model="models/text-embedding-004"),
            collection_name=collection_name,
            api_endpoint=os.getenv("ASTRA_DB_API_ENDPOINT"),
            token=os.getenv("ASTRA_DB_APPLICATION_TOKEN"),
            namespace= os.getenv("ASTRA_DB_KEYSPACE"),)

#Adding documnents
inserted_ids=vstore.add_documents(documents)

In [48]:
inserted_ids[0]

'da4f0950ea4448dd8675202464ea72d3'

In [49]:
len(inserted_ids)

450

In [50]:
vstore

In [51]:
#optionally do a quick search
query="Can you tell me the low budget headphone?"
results=vstore.similarity_search(query)

In [52]:
results

[Document(id='94c418e2262845ff9d9b5db156f7d0e8', metadata={'product_name': 'BoAt BassHeads 100 Wired Headset', 'product_rating': 4, 'product_summary': 'Very Good'}, page_content='l am happy with that purchase.... l like the design of the headphone.... l like the quality of the bass also...'),
 Document(id='8ff71a4e24634d4796c09fcc8817bf8d', metadata={'product_name': 'OnePlus Bullets Wireless Z Bass Edition Bluetooth Headset', 'product_rating': 5, 'product_summary': 'Wonderful'}, page_content='The quality of the headphone is awesome. Easy to connect to any mobile and PC. Sound quality and bass are satisfactory and worth every penny. Go for it!'),
 Document(id='4257a686061e4894a737641229ffae13', metadata={'product_name': 'OnePlus Bullets Wireless Z Bass Edition Bluetooth Headset', 'product_rating': 5, 'product_summary': 'Must buy!'}, page_content='Super super head phones.the sound quality is absolutely amazing.and battery life is good.'),
 Document(id='671a96e9534144ba8079e41acbc6c1f8', 

In [53]:
for res in results:
    print(f"Content: {res.page_content}\nMetadata: {res.metadata}\n")

Content: l am happy with that purchase.... l like the design of the headphone.... l like the quality of the bass also...
Metadata: {'product_name': 'BoAt BassHeads 100 Wired Headset', 'product_rating': 4, 'product_summary': 'Very Good'}

Content: The quality of the headphone is awesome. Easy to connect to any mobile and PC. Sound quality and bass are satisfactory and worth every penny. Go for it!
Metadata: {'product_name': 'OnePlus Bullets Wireless Z Bass Edition Bluetooth Headset', 'product_rating': 5, 'product_summary': 'Wonderful'}

Content: Super super head phones.the sound quality is absolutely amazing.and battery life is good.
Metadata: {'product_name': 'OnePlus Bullets Wireless Z Bass Edition Bluetooth Headset', 'product_rating': 5, 'product_summary': 'Must buy!'}

Content: It's a really perfect earphone. I love it . Sound quality and base is too good in this cost.
Metadata: {'product_name': 'BoAt BassHeads 100 Wired Headset', 'product_rating': 3, 'product_summary': 'Fair'}



In [68]:
#vectore store as retriever
retriever=vstore.as_retriever(search_kwargs={"k": 5})

In [60]:
retriever.invoke(query)

[Document(id='9ba51da89ccb4550b1e4c8edbec829d4', metadata={'product_name': 'OnePlus Bullets Wireless Z Bass Edition Bluetooth Headset', 'product_rating': 5, 'product_summary': 'Awesome'}, page_content='Best earphone'),
 Document(id='94c418e2262845ff9d9b5db156f7d0e8', metadata={'product_name': 'BoAt BassHeads 100 Wired Headset', 'product_rating': 4, 'product_summary': 'Very Good'}, page_content='l am happy with that purchase.... l like the design of the headphone.... l like the quality of the bass also...')]

In [63]:
PROMPT_TEMPLATES={
    "product_bot": """
    You are an expert EcommerceBot specialized in product recommendations and handling customer queries.
    Analyze the provided product titles, ratings, and reviews to provide accurate, helpful responses.
    Stay relevant to the context, and keep your answers concise and informative.

    CONTEXT:
    {context}

    QUESTION: {question}

    YOUR ANSWER:
    """

}

In [69]:
prompt=ChatPromptTemplate.from_template(PROMPT_TEMPLATES["product_bot"])
llm=gemini_model=ChatGroq(model="deepseek-r1-distill-llama-70b" ,api_key=os.getenv("GROQ_API_KEY"))

chain=(
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

query="Can you tell me the low budget headphone?"
output=chain.invoke(query)



In [70]:
output

'<think>\nOkay, so the user is asking for a low-budget headphone. Let me look through the provided documents to find the best options. \n\nFirst, I see the BoAt BassHeads 100 Wired Headset. It has a rating of 4 and one review mentions it\'s "very good" with great design and bass. Another review gives it 3 stars but still says it\'s perfect with good sound quality and bass for the cost. Since it\'s wired, it\'s likely more affordable.\n\nNext, the OnePlus Bullets Wireless Z Bass Edition Bluetooth Headset has a 5-star rating and positive reviews about sound quality, bass, and battery life. It\'s wireless, which might be a bit pricier, but the reviews say it\'s worth the money. \n\nThen there\'s the realme Buds Q Bluetooth Headset with a 5-star rating and a review highlighting very good bass at a price point of 1599. That seems like a good deal for wireless earbuds.\n\nConsidering budget, wired headphones are usually cheaper. The BoAt BassHeads 100 is a good wired option with positive fee

In [75]:
app=FastAPI()

# BASE_DIR=os.path.dirname(os.path.abspath(__file__))



app.mount("/static", StaticFiles(directory="D:/Customer_Support_System(GenAi-RAG)/static"))

templates=Jinja2Templates(directory="D:/Customer_Support_System(GenAi-RAG)/templates")

# Allow CORS (optional for frontend)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [77]:
@app.get("/", response_class=HTMLResponse)
async def index(request: Request):
    """
    Render the chat interface.
    """
    return templates.TemplateResponse("chat.html", {"request": request})



In [78]:
@app.post("/get",response_class=HTMLResponse)
async def chat(msg:str=Form(...)):
    result=chain.invoke(msg)
    print(f"Response: {result}")
    return result

In [80]:
import nest_asyncio
import uvicorn

if __name__ == "__main__":
    nest_asyncio.apply()
    uvicorn.run(app)

INFO:     Started server process [15624]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:57154 - "GET / HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "d:\Customer_Support_System(GenAi-RAG)\venv\lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "d:\Customer_Support_System(GenAi-RAG)\venv\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
  File "d:\Customer_Support_System(GenAi-RAG)\venv\lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "d:\Customer_Support_System(GenAi-RAG)\venv\lib\site-packages\starlette\applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "d:\Customer_Support_System(GenAi-RAG)\venv\lib\site-packages\starlette\middleware\errors.py", line 186, in __call__
    raise exc
  File "d:\Customer_Support_System(GenAi-RAG)\venv\lib\site-packages\starlette\midd

INFO:     127.0.0.1:57155 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:57156 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:57156 - "GET /openapi.json HTTP/1.1" 200 OK


d:\Customer_Support_System(GenAi-RAG)\venv\lib\site-packages\fastapi\openapi\utils.py:246: UserWarning: Duplicate Operation ID chat_get_post for function chat
  warnings.warn(message, stacklevel=1)


Response: <think>
Okay, so the user is asking about a low-budget headphone. Let me look at the context provided. There are a few products listed here.

First, the BoAt BassHeads 100 Wired Headset has a rating of 4 and is described as "Very Good." The reviews mention it's happy with the purchase, likes the design, quality, and bass. Another review gives it a 3 and says it's "Fair," but still mentions the sound quality and bass are good for the cost. So this seems like an affordable option.

Then there's the OnePlus Bullets Wireless Z Bass Edition Bluetooth Headset with a 5-star rating. The reviews use words like "awesome," "worth every penny," and mention good battery life. However, since it's from OnePlus, it might be a bit pricier than the BoAt option.

Lastly, the realme Buds Q Bluetooth Headset also has a 5-star rating and the review says it's a nice product at that range, specifically mentioning it was bought for 1599, which is a good price point.

So, considering the ratings and t

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "d:\Customer_Support_System(GenAi-RAG)\venv\lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "d:\Customer_Support_System(GenAi-RAG)\venv\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
  File "d:\Customer_Support_System(GenAi-RAG)\venv\lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "d:\Customer_Support_System(GenAi-RAG)\venv\lib\site-packages\starlette\applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "d:\Customer_Support_System(GenAi-RAG)\venv\lib\site-packages\starlette\middleware\errors.py", line 186, in __call__
    raise exc
  File "d:\Customer_Support_System(GenAi-RAG)\venv\lib\site-packages\starlette\midd